<a href="https://colab.research.google.com/github/rfmergarejo/Physics/blob/main/double_pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The double pendulum is a very complicate problem in physics where an analytical solution is almost impossible. 

In [ ]:
import numpy as np
import sympy as smp

# Integrator.... Differential equation solver
from scipy.integrate import odeint

# All those librarys is for visualization.
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import PillowWriter

The representation is the following:





Defining the apporpiate symbols using `sympy` 

In [4]:
t, g = smp.symbols('t, g')
m1, m2 = smp.symbols('m1, m2')
L1, L2 = smp.symbols('L1, L2')

In [8]:
thet1, thet2 = smp.symbols(r'\theta_1, \theta_2')

Explicitly writting $\theta_1$ and $\theta_2$ as a function of time.

In [10]:
thet1 = thet1(t)
thet2 = thet2(t)

TypeError: ignored